<a href="https://colab.research.google.com/github/snpsuen/Deep_Learning_Data/blob/main/RNN_GBP_HKD_Daily_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
# lstm for time series forecasting
import numpy
from numpy import sqrt
from numpy import asarray
from pandas import read_csv
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [93]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps, select):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix, select], sequence[end_ix, select]
		X.append(seq_x)
		y.append(seq_y)
	return asarray(X), asarray(y)

In [94]:
# load the dataset
path = 'https://raw.githubusercontent.com/snpsuen/Deep_Learning_Data/main/dataset/GBP_HKD_Historical_Data_20221201-20230303.csv'
df = read_csv(path, header=0, index_col=0, squeeze=True)
# extract price columns from loaded data frame
df_prices = df.iloc[:, 0:4]
# retrieve the values
values = df_prices.values.astype('float32')
# specify the window size
n_steps = 10
select = 0
# split into samples
X, y = split_sequence(values, n_steps, select)
print("X = ", X)
print("y = ", y)
print("X.shape = %s, y.shape = %s" %(X.shape, y.shape))
# reshape into [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))
# split into train/test
n_test = 10
X_train, X_test, y_train, y_test = X[:-n_test], X[-n_test:], y[:-n_test], y[-n_test:]
print("After reshape, X_train.shape = %s, X_test.shape = %s, y_train.shape = %s, y_test.shape = %s" %(X_train.shape, X_test.shape, y_train.shape, y_test.shape))
print("X_test = ", X_test)
print("y_test = ", y_test)
input_shape = X.shape[1:]
print("input_shape = ", input_shape)

X =  [[9.5229 9.5701 9.4667 9.4376 9.5049 9.5245 9.541  9.5415 9.6036 9.6546]
 [9.5701 9.4667 9.4376 9.5049 9.5245 9.541  9.5415 9.6036 9.6546 9.4689]
 [9.4667 9.4376 9.5049 9.5245 9.541  9.5415 9.6036 9.6546 9.4689 9.4492]
 [9.4376 9.5049 9.5245 9.541  9.5415 9.6036 9.6546 9.4689 9.4492 9.4512]
 [9.5049 9.5245 9.541  9.5415 9.6036 9.6546 9.4689 9.4492 9.4512 9.4857]
 [9.5245 9.541  9.5415 9.6036 9.6546 9.4689 9.4492 9.4512 9.4857 9.421 ]
 [9.541  9.5415 9.6036 9.6546 9.4689 9.4492 9.4512 9.4857 9.421  9.3873]
 [9.5415 9.6036 9.6546 9.4689 9.4492 9.4512 9.4857 9.421  9.3873 9.4102]
 [9.6036 9.6546 9.4689 9.4492 9.4512 9.4857 9.421  9.3873 9.4102 9.414 ]
 [9.6546 9.4689 9.4492 9.4512 9.4857 9.421  9.3873 9.4102 9.414  9.3769]
 [9.4689 9.4492 9.4512 9.4857 9.421  9.3873 9.4102 9.414  9.3769 9.3662]
 [9.4492 9.4512 9.4857 9.421  9.3873 9.4102 9.414  9.3769 9.3662 9.3959]
 [9.4512 9.4857 9.421  9.3873 9.4102 9.414  9.3769 9.3662 9.3959 9.4472]
 [9.4857 9.421  9.3873 9.4102 9.414  9.3769 9.

In [100]:
# Extract the last 9 data points from X_test to form X_predict
X_predict = X_test[-1][1:]
# Extract the last y_test data point
y_last = numpy.array([y_test[-1]])

# Append y_last to the X_predict
X_predict = numpy.append(X_predict, y_last)

# Reshape X_predict to (1, 10, 1)
X_predict = X_predict.reshape(1, X_predict.shape[0], 1)
print("X_predict = ", X_predict)
print("X_predict.shape = ", X_predict.shape)


X_predict =  [[[9.4308]
  [9.4997]
  [9.45  ]
  [9.4264]
  [9.372 ]
  [9.4614]
  [9.4346]
  [9.4406]
  [9.3731]
  [9.4037]]]
X_predict.shape =  (1, 10, 1)


In [96]:
# define model
model = Sequential()
# model.add(LSTM(100, activation='relu', kernel_initializer='he_normal', input_shape=input_shape))
model.add(LSTM(256, kernel_initializer='he_normal', input_shape=input_shape))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1))
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 256)               264192    
                                                                 
 dense_42 (Dense)            (None, 128)               32896     
                                                                 
 dense_43 (Dense)            (None, 64)                8256      
                                                                 
 dense_44 (Dense)            (None, 1)                 65        
                                                                 
Total params: 305,409
Trainable params: 305,409
Non-trainable params: 0
_________________________________________________________________


In [97]:
# compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# fit the model
model.fit(X_train, y_train, epochs=40, batch_size=32, verbose=2, validation_data=(X_test, y_test))

Epoch 1/40
2/2 - 4s - loss: 108.9638 - mae: 10.4324 - val_loss: 85.2663 - val_mae: 9.2339 - 4s/epoch - 2s/step
Epoch 2/40
2/2 - 0s - loss: 84.0908 - mae: 9.1670 - val_loss: 69.2657 - val_mae: 8.3225 - 117ms/epoch - 58ms/step
Epoch 3/40
2/2 - 0s - loss: 68.5388 - mae: 8.2755 - val_loss: 56.5133 - val_mae: 7.5174 - 98ms/epoch - 49ms/step
Epoch 4/40
2/2 - 0s - loss: 55.8569 - mae: 7.4697 - val_loss: 44.7502 - val_mae: 6.6895 - 117ms/epoch - 58ms/step
Epoch 5/40
2/2 - 0s - loss: 44.1967 - mae: 6.6437 - val_loss: 34.9925 - val_mae: 5.9153 - 123ms/epoch - 62ms/step
Epoch 6/40
2/2 - 0s - loss: 34.5496 - mae: 5.8746 - val_loss: 26.2639 - val_mae: 5.1247 - 112ms/epoch - 56ms/step
Epoch 7/40
2/2 - 0s - loss: 25.8390 - mae: 5.0784 - val_loss: 18.4851 - val_mae: 4.2993 - 103ms/epoch - 52ms/step
Epoch 8/40
2/2 - 0s - loss: 18.0676 - mae: 4.2444 - val_loss: 11.6340 - val_mae: 3.4107 - 103ms/epoch - 51ms/step
Epoch 9/40
2/2 - 0s - loss: 11.2780 - mae: 3.3475 - val_loss: 6.1603 - val_mae: 2.4817 - 100

In [98]:
# evaluate the model
mse, mae = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f, MAE: %.3f' % (mse, sqrt(mse), mae))

MSE: 0.002, RMSE: 0.040, MAE: 0.035


In [99]:
# make a prediction
## row = asarray([5, 1, 1, 2, 2]).reshape((1, n_steps, 1))
## print("row = %s" %(row))
## yhat = model.predict(row)
## print('Predicted: %.3f' % (yhat))
print("X_predict = %s" %(X_predict))
print("X_predict.shape = ", X_predict.shape)
# print("After reshape, X_train.shape = %s, X_test.shape = %s, X_predict.shape = %s, y_train.shape = %s, y_test.shape = %s, y_predict.shape = %s" %(X_train.shape, X_test.shape, X_predict.shape, y_train.shape, y_test.shape, y_predict.shape))
yhat = model.predict(X_predict)
print('Predicted: %.3f' % (yhat))

X_predict = [[[9.4308]
  [9.4997]
  [9.45  ]
  [9.4264]
  [9.372 ]
  [9.4614]
  [9.4346]
  [9.4406]
  [9.3731]
  [9.4037]]]
X_predict.shape =  (1, 10, 1)
1/1 [==============================] - 1s 559ms/step
Predicted: 9.416
